In [38]:
import pandas as pd

import torch
from torch import nn
from pyro.nn.auto_reg_nn import AutoRegressiveNN

In [34]:
x = torch.randn(100, 10)
arn = AutoRegressiveNN(10, [10], param_dims=[1])
p = arn(x)

In [35]:
p.shape

torch.Size([100, 10])

In [36]:
x.shape

torch.Size([100, 10])

In [33]:
pd.DataFrame(p.detach().numpy())

,0,1,2,3,4,5,6,7,8,9
0,-0.015917,0.248514,-0.316904,-0.269666,-0.259105,0.169129,-0.108173,-0.130347,-0.245178,0.015393
1,0.083069,0.337242,-0.365985,-0.164847,-0.259105,0.169129,-0.101057,-0.130347,-0.188734,0.020842
2,-0.097815,0.109435,-0.213368,-0.063219,-0.259105,0.169129,-0.021830,-0.130347,-0.100119,0.041720
3,0.027545,0.267006,-0.437566,-0.332692,-0.259105,0.169129,-0.184909,-0.130347,-0.217232,-0.021448
4,-0.040029,0.209996,-0.266993,-0.047321,-0.260747,0.169129,-0.039653,-0.130347,-0.156152,0.031274
...,...,...,...,...,...,...,...,...,...,...
95,-0.038624,0.204284,-0.192796,-0.267576,-0.259723,0.169129,-0.057952,-0.130347,-0.282300,0.042393
96,0.000190,0.254068,-0.260217,-0.047348,-0.259105,0.169129,-0.043020,-0.130347,-0.156761,0.044363
97,-0.027880,0.170119,-0.266572,-0.060666,-0.259105,0.169129,-0.016451,-0.130347,-0.105610,0.044363
98,0.007624,0.297084,-0.303005,-0.201916,-0.259105,0.169129,-0.085469,-0.130347,-0.257904,0.030525


In [39]:
class AR2_full(nn.Module): #returns entire sequence
    def __init__(self, data, p=2, init=None):
        super(AR2_full, self).__init__()
        self.p = p
        self.data = data.detach()
        self.data.requires_grad = False
        self.init = nn.Parameter(self.data[0:self.p])
        self.phi = nn.Parameter(to.randn(self.p)/ 10)
        self.const = nn.Parameter(to.randn(1)/10)
        self.var = nn.Parameter(to.tensor([1.]))
        self.lastx = to.randn(p)/10.
        N,p= self.data.shape[0], self.p
    
    def set_lastx(self,data): #used for initialization only
        with to.no_grad():
            for i in range(data.shape[0]):
                self.lastx[i] = data[i]
            
    def append_lastx(self,y_new):
        with to.no_grad():
            for i in range(self.lastx.shape[0]-1):
                self.lastx[i+1] = self.lastx[i]
            self.lastx[0] = y_new
    
    def loglikelihood(self):
        N,p = self.data.shape[0],self.p
        logprob = to.Tensor([0.])
        logprob.requires_grad=True
        for i in range(p,N,1):
            data = self.data[i-p:i]
            loc = self.phi @ data + self.const
            l = to.distributions.Normal(loc=loc,scale=self.var)
            logprob = logprob + l.log_prob(self.data[i])
        return logprob
    
    def forward(self):
        noise = to.distributions.Normal(loc=0,scale=self.var).sample()
        lastx = to.stack(list(self.lastx)).detach()
        y = (self.phi @ lastx) + noise + self.const
        self.append_lastx(y.detach())
        return y

In [40]:
ar2 = AR2_full(gendata3,p=2)
optim = to.optim.SGD(lr=1e-4,params=ar2.parameters())
losses = []

n_steps = 100
for step in range(n_steps):
    optim.zero_grad()
    with to.no_grad(): #reset initial values
        ar2.set_lastx(ar2.init)
    loss = -1*ar2.loglikelihood()
    loss.backward()
    losses.append(loss.detach().numpy())
    optim.step()
    cur_step = cur_step + 1 if cur_step < 3 else 0

NameError: name 'gendata3' is not defined

In [55]:
thing = nn.Linear(1, 1, bias=True)

In [56]:
x = torch.randn(10, 1)

In [57]:
thing(x)

tensor([[-0.0284],
        [ 0.9978],
        [-1.0957],
        [-0.5926],
        [-0.8894],
        [ 0.4381],
        [ 0.9547],
        [-1.0362],
        [ 0.3439],
        [ 1.8091]], grad_fn=<MmBackward>)

In [58]:
x

tensor([[-0.0367],
        [ 1.2887],
        [-1.4152],
        [-0.7654],
        [-1.1488],
        [ 0.5658],
        [ 1.2331],
        [-1.3384],
        [ 0.4442],
        [ 2.3365]])

In [59]:
for jj in thing.parameters():
    print(jj)

Parameter containing:
tensor([[0.7743]], requires_grad=True)
